In [3]:
import numpy as np 
import os
import cv2

# Importing Model
Use one of the model files in the models directory as model.py (default to mobilenetV2) 

In [4]:
from model import DepthEstimate
model = DepthEstimate()                   
batch_size     = 16                       
learning_rate  = 0.0001
epochs         = 6

/opt/conda/lib/python3.7/site-packages/keras_applications/mobilenet_v2.py:294: UserWarning: `input_shape` is undefined or non-square, or `rows` is not in [96, 128, 160, 192, 224]. Weights for input shape (224, 224) will be loaded as the default.
  warnings.warn('`input_shape` is undefined or non-square, '


9412608/9406464 [==============================] - 0s 0us/step
Base model loaded

Model created.


# Specifying the Loss Function 

In [8]:
import tensorflow.keras.backend as K

def depth_loss_function(y_true, y_pred, theta=0.3, maxDepthVal=1000.0/10.0):
    
    # Point-wise depth
    l_depth = K.mean(K.abs(y_pred - y_true), axis=-1)

    # Structural similarity (SSIM) index
    l_ssim = K.clip((1 - tf.image.ssim(y_true, y_pred, maxDepthVal)) * 0.5, 0, 1)

    # Weights
    w1 = 1.0
    w3 = theta

    return (w1 * l_ssim) + (w2 * K.mean(l_edges)) + (w3 * K.mean(l_depth))

In [9]:
import tensorflow
#from loss import depth_loss_function   

optimizer = tensorflow.keras.optimizers.Adam(lr=learning_rate, amsgrad=True)

model.compile(loss=depth_loss_function, optimizer=optimizer)

#  Loading weights
 Load weights available in the google drive corresponding to the model loaded 

In [13]:
model.load_weights("~/cp.ckpt")

# Predicting the DepthMap
Output is saved as out.png

In [ ]:
import matplotlib.image as mpimg 
import matplotlib.pyplot as plt 
  
# Read Images 
rgb = mpimg.imread('/path-to-input-image/')  # Load input image
rgb = rgb.resize((480,640))   #Resize Input image to 480x640
# Output Images 
plt.imshow(rgb)

In [ ]:
import scipy
import time
import matplotlib
a    = time.time()
img1 = rgb
img1 = img1.astype('float32')
out = predict(model,img1,10.0,1000.0)
out = out.reshape(240,320)
out = scipy.ndimage.zoom(out, 2, order=1)
plt.figure(figsize=(10,5))
plt.imshow(out)
plt.show()
matplotlib.image.imsave("~/out.png",out,cmap='gray')

# Segmentation 
We used the gluoncv library

In [ ]:
!pip install gluoncv --upgrade
!pip install -U --pre mxnet -f https://dist.mxnet.io/python/mkl
# if cuda 10.1 is installed
!pip install -U --pre mxnet -f https://dist.mxnet.io/python/cu100mkl

In [ ]:
import mxnet as mx
from mxnet import image
from mxnet.gluon.data.vision import transforms
import gluoncv
img = image.imread('path to input image')
# using cpu
ctx = mx.cpu(0)

from gluoncv.data.transforms.presets.segmentation import test_transform
img = test_transform(img, ctx)

model = gluoncv.model_zoo.get_model('deeplab_resnet101_ade', pretrained=True)
output = model.predict(img)
predict = mx.nd.squeeze(mx.nd.argmax(output, 1)).asnumpy()
plt.imshow(predict)

Copying the depthmap to out1

Taking minimum of the depths (min_depth) that are outside the segemtation area

In [ ]:
import numpy as np
#seg=np.clip(predict,np.mean(predict),np.max(predict))
#seg = predict < np.mean(predict) 
min_depth = np.max(out)
avg = np.mean(predict)
for i in range(len(predict)):
    for j in range(len(predict[0])):
        out1[i][j] = out[i][j]
        if predict[i][j]<avg:
            if out[i][j]<min_depth:
                min_depth=out[i][j]

The pixles inside the segmented area are divided by a certain factor to separate them from background and give a pop effect

In [ ]:
for i in range(len(predict)):
    for j in range(len(predict[0])):
        if predict[i][j]>avg:
                out1[i][j]= out1[i][j]/1.25
        else:
            out1[i][j]=out[i][j]

We can Also push all the pixels in the segmented area to "min_depth" to get the average-3D mode as discussed in demo (Please refer to 3D demo directory under Demos)

# RGB + DepthMap to PointClouds
We are using the open3d library here.
Final pointcloud is saved as pcd.ply

In [ ]:
!pip install open3d

In [ ]:
import open3d as o3d
out1=out1.astype('float32')
color = o3d.geometry.Image(rgb)                                   
depth = o3d.geometry.Image(out1)
rgbd_image = o3d.geometry.RGBDImage.create_from_color_and_depth(color, depth,convert_rgb_to_intensity=False)
print(rgbd_image)

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
plt.subplot(1, 2, 1)
plt.title('RGB image')
plt.imshow(rgbd_image.color)
plt.subplot(1, 2, 2)
plt.title('depth image')
plt.imshow(rgbd_image.depth)
plt.show()

In [ ]:
pcd = o3d.geometry.PointCloud.create_from_rgbd_image(
        rgbd_image,
        o3d.camera.PinholeCameraIntrinsic(
            o3d.camera.PinholeCameraIntrinsicParameters.PrimeSenseDefault))

pcd.transform([[1, 0, 0, 0], [0, -1, 0, 0], [0, 0, -1, 0], [0, 0, 0, 1]])
o3d.io.write_point_cloud("~/pcd.ply", pcd)
o3d.visualization.draw_geometries([pcd],window_name='a')